Christian Sutton   
10/10/20    
Module 6  

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
SRdata = pd.read_csv("C:/Users/physi/Desktop/AppliedMachineLearning_EN.705.601/Mod3/master.csv")

# Problem 1
**[20 pts] What is the dependent variable you decided? Why?**   
The dependent variable selected for this case was suicides/100k pop because it represents the number of suicides corrected by a counrty's population size awhich allows countries to be compared. 


# Problem 2  
**[20 pts] Set the dependent variable into two categories based on a defensible criteria. (Hint: skirts of the probability density function)**  

I have taken "suicides/100k pop" and binned it into two groups to build a dependent variable. One group is 17.73 per 100K and less and the other group is greater than 17.73 per 100K, where 17.73 represents the the top quartile of suicides per 100k. The top quartile seems like a reasonable threshold for distingishing a high rate of suicide. 

# Problem 3  
**[20 pts] Develop your classification model(s) to solve your defined problem.**   


In [2]:
#I am removing countries that have been contributing repeated zeros to the data set

SRdata = SRdata[SRdata.country != "Antigua and Barbuda"]
SRdata = SRdata[SRdata.country != "Aruba"]
SRdata = SRdata[SRdata.country != "Bosnia and Herzegovina"]
SRdata = SRdata[SRdata.country != "Cabo Verdea"]
SRdata = SRdata[SRdata.country != "Cyprus"]
SRdata = SRdata[SRdata.country != "Dominica"]
SRdata = SRdata[SRdata.country != "Grenada"]
SRdata = SRdata[SRdata.country != "Jamaica"]
SRdata = SRdata[SRdata.country != "Kuwait"]
SRdata = SRdata[SRdata.country != "Maldives"]
SRdata = SRdata[SRdata.country != "Montenegro"]
SRdata = SRdata[SRdata.country != "Oman"]
SRdata = SRdata[SRdata.country != "Qatar"]
SRdata = SRdata[SRdata.country != "Saint Kitts and Nevis"]

In [3]:
#Since HDI for year -  is missing most of it's values, this feature will be removed
#Country-year -  is redundant to features "year" and "country"
#gdp_for_year - removing this feature because it isn't used later
#removing counntry, year and gdp_per_capita, due to low correlation
#population and suicides_no are being removed due to unwanted correlation between features

SRdata =SRdata.drop(["HDI for year","country-year"," gdp_for_year ($) ","country","year",\
                     "gdp_per_capita ($)","population","suicides_no"], axis =1)

In [4]:
#binning my dependent variable

cut_labels = ['Low chance', 'High Chance']
cut_bins = [-.1, 17.73, 2000]
SRdata['Chance of suicide'] = pd.cut(SRdata['suicides/100k pop'], bins=cut_bins, labels=cut_labels)

In [5]:
# encode ordinal values to age, suicide chance, 
ages = {'15-24 years':20, '5-14 years':10,'25-34 years':30,'35-54 years':45,'55-74 years':65,'75+ years':75}
Generations = {'G.I. Generation':1,'Silent':2,'Boomers':3,\
               'Generation X':4,'Millenials':5,'Generation Z':6}
Chance= {'High Chance':1,'Low chance':0}
SRdata['age']= SRdata['age'].map(ages)
SRdata['generation']= SRdata['generation'].map(Generations)
SRdata['Chance of suicide']= SRdata['Chance of suicide'].map(Chance)

In [6]:
#OneHotEncode sex
SRdata =pd.get_dummies(SRdata)
SRdata =SRdata.drop(["sex_female","suicides/100k pop"],axis =1)

In [7]:
 SRdata.head()

,age,generation,Chance of suicide,sex_male
0,20,4,0,1
1,45,2,0,1
2,20,4,0,0
3,75,1,0,1
4,30,3,0,1


# Problem 4  
**[20 pts] Evaluate (and report) the model performance(s) using some of the standard techniques (e.g. 80-20 split, 10-fold cross validation, etc.).**  


In [8]:
#spilt independent and dependent variables 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier


# 80/20 split
X, Y =SRdata.iloc[:,[0,1,3]], SRdata.iloc[:,2]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=.2,random_state=0)

pipe_l1 = make_pipeline(StandardScaler(),
                            RandomForestClassifier(5,random_state=10))
pipe_l1.fit(X_train, y_train)
y_pred = pipe_l1.predict(X_test)

In [9]:
#reshape
y_pred =y_pred.reshape((5162,1))
y_test = np.array(y_test)
y_test= y_test.reshape((5162,1))

# make new dataframe
Results = pd.DataFrame(data=y_pred, columns= ["Predicted"])
Results["Actual"]= y_test

In [10]:
Results.head()

,Predicted,Actual
0,0,0
1,0,0
2,0,0
3,1,1
4,1,1


In [11]:
from sklearn.metrics import f1_score,accuracy_score

print(f'F1 Score ={f1_score(Results.iloc[:,1],Results.iloc[:,0]):.2f}  \
Accuracy ={accuracy_score(Results.iloc[:,1],Results.iloc[:,0]):.2f} ')

F1 Score =0.66  Accuracy =0.82 


# Problem 5  
**[20 pts] Using your classifier model, what is the predicted category of your dependent variable for the input: "year=2000, generation=Generation X, age=20, gender=male"?**  


In [14]:
answer1 = pipe_l1.predict(np.array([20,4,1]).reshape(1,-1))

In [19]:
print(f' The answer of {answer1} means the datapoint represents a low risk of suicide in this model')

 The answer of [0] means the datapoint represents a low risk of suicide in this model


# problem 6  
**[20 pts bonus] Using your (perhaps a different?) model, what is the actual probability of a
"Generation X 20-year-old male living in a country with 40000 gdp_per_capita" would commit suicide?**  



In [21]:
answer2 = pipe_l1.predict_proba(np.array([20,4,1]).reshape(1,-1))

In [26]:
print(f' The answer of {answer2} means the datapoint represents a probability of 68% class 0 (low risk)')
print(f' and 31% probability of class 1 (high risk of suicide) in this model')

 The answer of [[0.68720502 0.31279498]] means the datapoint represents a probability of 68% class 0 (low risk)
 and 31% probability of class 1 (highrisk of suicide) in this model
